<div style='float: right'><img src='pic/wall.png'/></div>

## <div id='wall' />ウォールロジック

In [ ]:
#!pip3 install pulp ortoolpy
import pandas as pd
from itertools import product
from pulp import LpProblem, lpSum, lpDot, value
from ortoolpy import addvars, addbinvars
data = """\
4..1..
.4..2.
..2..2
1..1..
.1..1.
..3..2""".split()

### 問題
* 数字が記入されているマスからその数字の数だけ縦と横に線を引きます
* 1つのマスには1本の線しか引くことができません
* 数字が記入されているマスには線を引くことができません

### 変数
* v：各位置のどの方向か  (1)
* r：各位置の方向別長さ  (2)

### 制約
* 数字があれば、方向別長さの和に等しく、かつその位置に矢印がないこと  (3)
* 数字がなければ矢印は1方向のみ  (4)
* 数字がなければ矢印の方向に長さを1足すこと  (5)

In [ ]:
ni, nj = len(data), len(data[0])
mx = max(ni, nj)
a = pd.DataFrame([(i,j,k) for i,j,k in product(range(ni),
    range(nj),range(4))], columns=list('行列向'))
a['VDir'] = addbinvars(len(a))
a['VLen'] = addvars(len(a))
a[:2]

In [ ]:
drc = [(-1, 0, 0), (0, -1, 1), (0, 1, 2), (1, 0, 3)]
m = LpProblem()
for i,j in product(range(ni), range(nj)):
    b = a[(a.行==i)&(a.列==j)]
    if data[i][j].isdigit():
        m += lpSum(a[(a.行==i+y)&(a.列==j+x)&(a.向==k)].VLen
                   for y,x,k in drc) == int(data[i][j]) # (1)
        m += lpSum(b.VLen) == 0 # (1)
        continue
    m += lpSum(b.VDir) == 1 # (2)
    for y,x,k in drc:
        r = b[b.向==k].iloc[0]
        m += r.VLen <= mx * r.VDir # (3)
        m += r.VLen <= r.VDir + lpSum(
            a[(a.行==i+y)&(a.列==j+x)&(a.向==k)].VLen) # (3)
m.solve()

In [ ]:
a['Val'] = a.VDir.apply(value)
print('\n'.join(''.join(' '+(data[i][j] if data[i][j].isdigit()
    else '↑←→↓'[int(value(lpDot([0,1,2,3],a[(a.行==i)&(a.列==j)].Val)))])
    for j in range(ni)) for i in range(nj)))